In [ ]:
from bach_open_taxonomy import ObjectivFrame
import bach
from sklearn import cluster

In [ ]:
of = ObjectivFrame.from_objectiv_data(start_date='2022-01-04',time_aggregation='YYYY-MM-DD')

In [ ]:
of['root'] = of.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id').fillna('empty')

In [ ]:
features = of[(of.event_type=='PressEvent')].groupby(['user_id','root']).session_hit_number.count()

In [ ]:
features.head()

In [ ]:
features_unstacked = features.unstack(fill_value=0).drop(columns=['empty'])
# sample or not
kmeans_frame = features_unstacked
kmeans_frame = features_unstacked.get_sample(table_name='kmeans_test',sample_percentage=50,overwrite=True,seed=2224)
kmeans_frame.count().head()

In [ ]:
# export to pandas now
df = kmeans_frame.to_pandas()

In [ ]:
df

In [ ]:
# do basic kmeans
est = cluster.KMeans(n_clusters=3)
est.fit(df)
df['cluster'] = est.labels_

In [ ]:
kmeans_frame['cluster'] = df['cluster']

In [ ]:
kmeans_frame.sort_values(['cluster','docs']).head(100)

In [ ]:
of_with_cluster = of.merge(kmeans_frame.cluster, on='user_id')

In [ ]:
of_with_cluster.head()

In [ ]:
for i in range(3):
    model_hub_results = of_with_cluster[of_with_cluster.cluster==i].mh.agg.session_duration(time_aggregation='YYYY')
    print(f"cluster {i}: {model_hub_results.to_pandas()[0]}\n")